## Creates binarized Seurat objects, integrates conditions and annotates genes by nearby peaks

In [1]:
# Input info
cellr_in = "/home/isshamie/data/Projects/Mito_Trace/output/mtscATAC/data/jan21_2021/MTblacklist/" 
outdir = "/home/isshamie/data/Projects/Mito_Trace/output/annotation/data/cd34norm/MTblacklist/mergedSamples/"

sample_names = "Control,Flt3l"
samples = "P2,J2"

# Parameters
nTop = 25000
cores = 24

In [2]:
library(repr)
options(repr.plot.width=12, repr.plot.height=12)

In [3]:
samples <- unlist(strsplit(samples, ",")[[1]])
sample_names <- unlist(strsplit(sample_names, ","))

samples

[1] "P2" "J2"

In [6]:
# install.packages("cowplot")
# install.packages("data.table")
# install.packages("dplyr")
# install.packages("BiocManager")
# BiocManager::install(c("GenomicRanges", "Signac", "EnsDb.Hsapiens.v75"))



Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Installing package(s) 'BiocVersion', 'GenomicRanges', 'Signac',
  'EnsDb.Hsapiens.v75'

also installing the dependencies ‘filelock’, ‘MatrixGenerics’, ‘DelayedArray’, ‘lambda.r’, ‘futile.options’, ‘gtools’, ‘caTools’, ‘sitmo’, ‘BiocFileCache’, ‘SummarizedExperiment’, ‘rjson’, ‘futile.logger’, ‘snow’, ‘BH’, ‘crosstalk’, ‘here’, ‘gplots’, ‘gridExtra’, ‘RcppArmadillo’, ‘spatstat.data’, ‘spatstat.utils’, ‘spatstat.sparse’, ‘abind’, ‘tensor’,

In [10]:
#BiocManager::install("multtest")


'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Installing package(s) 'multtest'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'fansi', 'glmnet', 'knitr', 'later'



In [11]:
#BiocManager::install("metap")


'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Installing package(s) 'metap'

also installing the dependency 'mutoss'


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'fansi', 'glmnet', 'knitr', 'later'



In [4]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library(dplyr)
library(future)

plan("multiprocess", workers = cores)
options(future.globals.maxSize = 8000 * 1024^2)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
#plan("multiprocess", workers = workers)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

## Merge all peaks

In [5]:
read.peaks <- function(exp, cellr_in){
    print('here')
    print(file.path(cellr_in, exp, "outs", "filtered_peak_bc_matrix", "peaks.bed"))
    peaks <- read.table(
      file = file.path(cellr_in, exp, "outs", "filtered_peak_bc_matrix", "peaks.bed"),
      col.names = c("chr", "start", "end")
    )
    # convert to genomic ranges
    gr <- makeGRangesFromDataFrame(peaks)
    return(gr)
}


gr.full <- c(sapply(samples, read.peaks, cellr_in=cellr_in, USE.NAMES=F))

gr.full.c <- gr.full[[1]]
if (length(gr.full)>1){
    for (i in 2:length(gr.full)){
      gr.full.c <- c(gr.full.c, gr.full[[i]])
    }
}
combined.peaks <- reduce(x = c(gr.full.c))

# Filter out bad peaks based on length
peakwidths <- width(combined.peaks)
combined.peaks <- combined.peaks[peakwidths  < 10000 & peakwidths > 20]
head(combined.peaks)

[1] "here"
[1] "/home/isshamie/data/Projects/Mito_Trace/output/mtscATAC/data/jan21_2021/MTblacklist//P2/outs/filtered_peak_bc_matrix/peaks.bed"
[1] "here"
[1] "/home/isshamie/data/Projects/Mito_Trace/output/mtscATAC/data/jan21_2021/MTblacklist//J2/outs/filtered_peak_bc_matrix/peaks.bed"


GRanges object with 6 ranges and 0 metadata columns:
      seqnames        ranges strand
         <Rle>     <IRanges>  <Rle>
  [1]     chr1    9942-10364      *
  [2]     chr1 191685-191736      *
  [3]     chr1 267780-268257      *
  [4]     chr1 271072-271548      *
  [5]     chr1 585995-586411      *
  [6]     chr1 605386-605767      *
  -------
  seqinfo: 24 sequences from an unspecified genome; no seqlengths

## Create fragment objects

In [6]:
allSE = c() 

samples_df <- cbind(sample_names, samples)
print('samples_df')
print(samples_df)
for (row in 1:nrow(samples_df)){
    exp <- (samples_df[[row, "samples"]])
    name <- (samples_df[[row, "sample_names"]]) 
    
#for (exp in samples) {
    print('exp')
    print(exp)
    print('name')
    print(name)
    barcode_path <- file.path(cellr_in, exp, "outs", "filtered_peak_bc_matrix", "barcodes.tsv")    
    barcodes <- readr::read_tsv(barcode_path, col_names = F) # %>% tidyr::unite(barcode)
    barcodes <- as.data.frame(barcodes) %>%  tibble::column_to_rownames(var="X1") %>% tibble::add_column(proj=name)
    frag_file <- file.path(cellr_in, exp, "outs", "fragments.tsv.gz")
    
    cells.meta.f <- file.path(cellr_in, exp, "outs", "singlecell.csv") 
    cells.meta <- as.data.frame(readr::read_csv(cells.meta.f)) %>% tibble::column_to_rownames(var="barcode") %>% tibble::add_column(proj=name)
    cells.meta <- cells.meta[rownames(cells.meta) %in% rownames(barcodes), ]

    # quantify multiome peaks in the scATAC-seq dataset
    
    
    print("Creating fragments object")
    frags.curr <- CreateFragmentObject(path = frag_file, cells= rownames(barcodes))
    #print(frags.curr)
    print("Quantifying peaks")
    ## Quantify peaks
    curr.counts <- FeatureMatrix(
      fragments = frags.curr,
      features = combined.peaks,
      cells = rownames(barcodes),
      process_n = cores
    )
    
    print("Creating chromatin assay")
    ## Create the objects and use simple filters
    curr_assay <- CreateChromatinAssay(curr.counts, fragments = frags.curr, min.cells = 10, min.features = 200)
    curr <- CreateSeuratObject(curr_assay, assay = "ATAC", project=name, meta.data=cells.meta)
    print('curr_assay')
    print(head(curr_assay))
    print('curr')
    print(head(curr[[]]))
    allSE = c(allSE, curr)
    #return(curr)
}

allSE

#allSE <- sapply(samples, create_frag, cellr_in=cellr_in)

[1] "samples_df"
     sample_names samples
[1,] "Control"    "P2"   
[2,] "Flt3l"      "J2"   
[1] "exp"
[1] "P2"
[1] "name"
[1] "Control"


Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom
Rows: 6875 Columns: 1

── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): X1


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 332219 Columns: 18

── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): barcode, cell_id
dbl (16): total, duplicate, chimeric, unmapped, lowmapq, mitochondrial, pass...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to qui

[1] "Creating fragments object"


Computing hash



[1] "Quantifying peaks"


...

Extracting reads overlapping genomic regions



[1] "Creating chromatin assay"


Warning message in CreateSeuratObject.Assay(curr_assay, assay = "ATAC", project = name, :
"Some cells in meta.data not present in provided counts matrix."


[1] "curr_assay"
data frame with 0 columns and 10 rows
[1] "curr"
                   orig.ident nCount_ATAC nFeature_ATAC total duplicate
AAACGAAAGAGCTCCC-1      Flt3l        4716          4430 25798      3956
AAACGAAAGCGATACG-1      Flt3l        2136          2066 18340      2478
AAACGAAAGGCTTCGC-1      Flt3l        3875          3691 28417      4398
AAACGAAAGTACAACA-1      Flt3l        1353          1317 12018      1524
AAACGAACAACGTACT-1      Flt3l        2973          2864 37690      3034
AAACGAACAAGCGGTA-1      Flt3l        1292          1257  9381      1416
                   chimeric unmapped lowmapq mitochondrial passed_filters
AAACGAAAGAGCTCCC-1      111       95    1643          4740          15253
AAACGAAAGCGATACG-1       78       92     794          9325           5573
AAACGAAAGGCTTCGC-1      132      122    1771         10224          11770
AAACGAAAGTACAACA-1       60       45     585          5688           4116
AAACGAACAACGTACT-1       82      125    1680         24974  

[[1]]
An object of class Seurat 
147518 features across 6543 samples within 1 assay 
Active assay: ATAC (147518 features, 0 variable features)

[[2]]
An object of class Seurat 
149495 features across 11949 samples within 1 assay 
Active assay: ATAC (149495 features, 0 variable features)


In [7]:
saveRDS(allSE, file.path(outdir, paste0("allSamples.rds")))

In [8]:
allSE

[[1]]
An object of class Seurat 
147518 features across 6543 samples within 1 assay 
Active assay: ATAC (147518 features, 0 variable features)

[[2]]
An object of class Seurat 
149495 features across 11949 samples within 1 assay 
Active assay: ATAC (149495 features, 0 variable features)


In [9]:
allSE[[1]][[]]

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,⋯,is__cell_barcode,TSS_fragments,DNase_sensitive_region_fragments,enhancer_region_fragments,promoter_region_fragments,on_target_fragments,blacklist_region_fragments,peak_region_fragments,peak_region_cutsites,proj
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
AAACGAAAGAGGTCCA-1,Control,1400,1357,16929,2587,58,70,674,9629,3911,⋯,1,740,0,0,0,740,0,1330,2434,Control
AAACGAAAGCGATACG-1,Control,3692,3505,45359,10546,184,184,2302,22247,9896,⋯,1,1611,0,0,0,1611,0,3462,6167,Control
AAACGAAAGTCGTGAG-1,Control,1031,994,10177,2450,48,60,657,2135,4827,⋯,1,775,0,0,0,775,0,966,1768,Control
AAACGAACAATAGTGA-1,Control,2829,2687,22452,4364,100,83,897,9233,7775,⋯,1,1239,0,0,0,1239,0,2701,4951,Control
AAACGAACACAATAAG-1,Control,1206,1184,11763,2003,43,57,680,5547,3433,⋯,1,621,0,0,0,621,0,1138,2019,Control
AAACGAACACTGATAC-1,Control,968,954,9449,1971,59,21,489,3785,3124,⋯,1,517,0,0,0,517,0,878,1595,Control
AAACGAACAGTAACCT-1,Control,702,693,6196,1097,19,17,288,2635,2140,⋯,1,397,0,0,0,397,0,651,1170,Control
AAACGAACATCGTGAT-1,Control,7716,6800,100024,36721,588,531,5317,36828,20039,⋯,1,3467,0,0,0,3467,0,7263,13261,Control
AAACGAAGTAACGGTG-1,Control,1111,1074,8454,1950,24,33,354,3729,2364,⋯,1,451,0,0,0,451,0,1042,1920,Control


## QC Metrics

In [21]:
library(Rsamtools)

qc <- function(se){
    # extract gene annotations from EnsDb
    annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v75)

    # change to UCSC style since the data was mapped to hg19
    seqlevelsStyle(annotations) <- 'NCBI'
    genome(annotations) <- "hg38"
    # add the gene information to the object
    Annotation(se) <- annotations
    gene.activities <- GeneActivity(se)
    # compute nucleosome signal score per cell
    se <- NucleosomeSignal(object = se)
    
    Annotation(se) <- annotations
    # compute TSS enrichment score per cell
    se <- TSSEnrichment(object = se, fast = FALSE)

    # add blacklist ratio and fraction of reads in peaks
    se$pct_reads_in_peaks <- se$peak_region_fragments / se$passed_filters * 100
    se$blacklist_ratio <- se$blacklist_region_fragments / se$peak_region_fragments
    se$high.tss <- ifelse(se$TSS.enrichment > 2, 'High', 'Low')
    se$nucleosome_group <- ifelse(se$nucleosome_signal > 4, 'NS > 4', 'NS < 4')

    return(se)
}
vPlot <- function(se){
      vPlot <- VlnPlot(
      object = se,
      features = c('pct_reads_in_peaks', 'peak_region_fragments',
                   'TSS.enrichment', 'blacklist_ratio', 'nucleosome_signal'),
      pt.size = 0.1,
      ncol = 5
    )  
    vPlot <- vPlot +    # Create grid of plots with title
             plot_annotation(title = se$orig.ident[[1]]) & 
             theme(plot.title = element_text(hjust = 0.5, size=15))
    #print(vPlot)
    return(vPlot)
}


Loading required package: Biostrings

Loading required package: XVector


Attaching package: 'Biostrings'


The following object is masked from 'package:base':

    strsplit




In [24]:
allSE <- lapply(allSE, qc)
saveRDS(allSE, file.path(outdir, paste0("allSamples.rds")))
lapply(allSE,vPlot)
ggsave(file.path(outdir, paste0("QC_01.png")))
ggsave(file.path(outdir, paste0("QC_01.pdf")))

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

In [ ]:
sessionInfo()